# 📊 意味カテゴリ画像分類研究 - Colab版

WordNetベースの意味カテゴリ画像分類システムの研究をGoogle Colabで実行します。

## 1. 環境セットアップ

In [ ]:
# Colab環境確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colabで実行中")
except:
    IN_COLAB = False
    print("❌ ローカル環境で実行中")

In [ ]:
# 必要なライブラリのインストール
if IN_COLAB:
    !pip install -q torch torchvision torchaudio
    !pip install -q transformers
    !pip install -q ultralytics
    !pip install -q opencv-python
    !pip install -q pillow
    !pip install -q matplotlib
    !pip install -q seaborn
    !pip install -q pandas
    !pip install -q numpy
    !pip install -q nltk
    !pip install -q scipy
    !pip install -q scikit-learn
    print("✅ ライブラリインストール完了")

In [ ]:
# 基本ライブラリのインポート
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import CLIPProcessor, CLIPModel
from ultralytics import YOLO
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import nltk
import seaborn as sns
from scipy import stats
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

print("✅ ライブラリインポート完了")

In [ ]:
# NLTK データのダウンロード
import nltk
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
from nltk.corpus import wordnet as wn
print("✅ WordNet準備完了")

In [ ]:
# Google Driveマウント（オプション）
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive接続完了")

## 2. 研究システムの初期化

In [ ]:
# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用デバイス: {device}")

# 作業ディレクトリ設定
import os
work_dir = '/content/research' if IN_COLAB else './research'
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)
print(f"作業ディレクトリ: {work_dir}")

In [ ]:
# モデルの初期化
print("🤖 AIモデルを初期化中...")

# BLIP (画像キャプション生成)
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# CLIP (画像-テキスト分類)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# YOLO (物体検出)
yolo_model = YOLO('yolov8n.pt')

print("✅ AIモデル初期化完了")

## 3. 研究用画像分類システム

In [ ]:
# 意味カテゴリ定義
SEMANTIC_CATEGORIES = {
    'person': ['person', 'man', 'woman', 'child', 'people', 'human', 'face'],
    'animal': ['dog', 'cat', 'bird', 'horse', 'cow', 'elephant', 'bear', 'zebra'],
    'food': ['pizza', 'hamburger', 'cake', 'apple', 'banana', 'sandwich', 'donut'],
    'vehicle': ['car', 'bus', 'truck', 'motorcycle', 'bicycle', 'airplane', 'boat'],
    'building': ['house', 'building', 'bridge', 'castle', 'church', 'tower'],
    'furniture': ['chair', 'table', 'sofa', 'bed', 'desk', 'bench'],
    'plant': ['tree', 'flower', 'grass', 'plant', 'garden', 'forest'],
    'landscape': ['mountain', 'beach', 'lake', 'river', 'sky', 'sunset']
}

print("✅ 意味カテゴリ定義完了")

In [ ]:
class SemanticImageClassifier:
    def __init__(self):
        self.results = []
        
    def generate_caption(self, image):
        """画像キャプションを生成"""
        inputs = blip_processor(image, return_tensors="pt").to(device)
        out = blip_model.generate(**inputs, max_length=50)
        caption = blip_processor.decode(out[0], skip_special_tokens=True)
        return caption
    
    def detect_semantic_category(self, caption):
        """意味カテゴリを検出"""
        caption_lower = caption.lower()
        
        # 各カテゴリとの一致度を計算
        category_scores = {}
        for category, keywords in SEMANTIC_CATEGORIES.items():
            score = sum(1 for keyword in keywords if keyword in caption_lower)
            if score > 0:
                category_scores[category] = score
        
        if category_scores:
            return max(category_scores, key=category_scores.get)
        return 'general'
    
    def classify_with_clip(self, image, labels, prefix=""):
        """CLIP による分類"""
        text_labels = [f"{prefix}{label}" for label in labels]
        inputs = clip_processor(text=text_labels, images=image, return_tensors="pt", padding=True).to(device)
        
        with torch.no_grad():
            outputs = clip_model(**inputs)
            logits_per_image = outputs.logits_per_image
            probs = logits_per_image.softmax(dim=1)
            
        max_prob_idx = probs.argmax().item()
        confidence = probs[0][max_prob_idx].item()
        predicted_label = labels[max_prob_idx]
        
        return predicted_label, confidence
    
    def get_specialized_approach(self, category):
        """カテゴリ特化アプローチを取得"""
        specialized_prompts = {
            'person': 'a photo of a person: ',
            'animal': 'a photo of an animal: ',
            'food': 'a photo of food: ',
            'vehicle': 'a photo of a vehicle: ',
            'building': 'a photo of architecture: ',
            'furniture': 'a photo of furniture: ',
            'plant': 'a photo of a plant: ',
            'landscape': 'a photo of a landscape: '
        }
        return specialized_prompts.get(category, '')
    
    def process_image(self, image_path):
        """画像を処理して研究データを収集"""
        # 画像読み込み
        image = Image.open(image_path).convert('RGB')
        
        # 1. キャプション生成
        caption = self.generate_caption(image)
        
        # 2. 意味カテゴリ検出
        semantic_category = self.detect_semantic_category(caption)
        
        # 3. YOLO物体検出
        yolo_results = yolo_model(image_path)
        detected_objects = []
        for r in yolo_results:
            for box in r.boxes:
                class_id = int(box.cls[0])
                class_name = yolo_model.names[class_id]
                confidence = float(box.conf[0])
                detected_objects.append({'class': class_name, 'confidence': confidence})
        
        # 4. 分類実験
        if semantic_category in SEMANTIC_CATEGORIES:
            category_labels = SEMANTIC_CATEGORIES[semantic_category]
            
            # 汎用アプローチ
            general_label, general_confidence = self.classify_with_clip(image, category_labels)
            
            # 特化アプローチ
            specialized_prefix = self.get_specialized_approach(semantic_category)
            specialized_label, specialized_confidence = self.classify_with_clip(
                image, category_labels, specialized_prefix
            )
            
            # 改善率計算
            improvement_rate = (specialized_confidence - general_confidence) / general_confidence * 100
        else:
            general_label = general_confidence = specialized_label = specialized_confidence = None
            improvement_rate = 0
        
        # 結果記録
        result = {
            'image_path': image_path,
            'caption': caption,
            'semantic_category': semantic_category,
            'detected_objects': detected_objects,
            'general_label': general_label,
            'general_confidence': general_confidence,
            'specialized_label': specialized_label,
            'specialized_confidence': specialized_confidence,
            'improvement_rate': improvement_rate
        }
        
        self.results.append(result)
        return result

# 分類器初期化
classifier = SemanticImageClassifier()
print("✅ 画像分類システム準備完了")

## 4. 画像アップロードと処理

In [ ]:
# 画像アップロード
if IN_COLAB:
    from google.colab import files
    print("📸 研究用画像をアップロードしてください（複数選択可能）:")
    uploaded = files.upload()
    
    uploaded_files = list(uploaded.keys())
    print(f"✅ {len(uploaded_files)}個の画像がアップロードされました")
    
    for filename in uploaded_files:
        print(f"  - {filename}")
else:
    # ローカル環境用のサンプル
    uploaded_files = ['sample1.jpg', 'sample2.jpg']  # 実際のファイルパスに変更
    print("ローカル環境: サンプル画像を使用")

In [ ]:
# 画像処理実行
print("🔬 画像分析を開始...")

for i, filename in enumerate(uploaded_files):
    print(f"\n📊 処理中 ({i+1}/{len(uploaded_files)}): {filename}")
    
    try:
        result = classifier.process_image(filename)
        
        # 結果表示
        print(f"  📝 キャプション: {result['caption']}")
        print(f"  📂 意味カテゴリ: {result['semantic_category']}")
        print(f"  🎯 検出物体数: {len(result['detected_objects'])}")
        
        if result['general_confidence']:
            print(f"  📈 汎用アプローチ: {result['general_label']} ({result['general_confidence']:.3f})")
            print(f"  🚀 特化アプローチ: {result['specialized_label']} ({result['specialized_confidence']:.3f})")
            print(f"  📊 改善率: {result['improvement_rate']:.2f}%")
        
        # 画像表示
        img = Image.open(filename)
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"{filename}\n{result['caption']}")
        plt.axis('off')
        plt.show()
        
    except Exception as e:
        print(f"  ❌ エラー: {e}")

print(f"\n✅ 全{len(uploaded_files)}画像の処理完了")

## 5. 研究結果の分析

In [ ]:
# 結果をDataFrameに変換
if classifier.results:
    df = pd.DataFrame(classifier.results)
    
    # 基本統計
    print("📊 研究結果統計:")
    print(f"  - 処理画像数: {len(df)}")
    print(f"  - 検出カテゴリ数: {df['semantic_category'].nunique()}")
    
    # 改善率の統計
    valid_improvements = df[df['improvement_rate'] != 0]['improvement_rate']
    if len(valid_improvements) > 0:
        print(f"  - 平均改善率: {valid_improvements.mean():.2f}%")
        print(f"  - 改善率標準偏差: {valid_improvements.std():.2f}%")
        print(f"  - 最大改善率: {valid_improvements.max():.2f}%")
        print(f"  - 最小改善率: {valid_improvements.min():.2f}%")
    
    # カテゴリ別統計
    print("\n📈 カテゴリ別分析:")
    category_stats = df.groupby('semantic_category').agg({
        'improvement_rate': ['count', 'mean', 'std'],
        'general_confidence': 'mean',
        'specialized_confidence': 'mean'
    }).round(3)
    
    print(category_stats)
    
    # 詳細結果表示
    print("\n📋 詳細結果:")
    display_df = df[['image_path', 'semantic_category', 'general_confidence', 
                     'specialized_confidence', 'improvement_rate']].round(3)
    print(display_df)
    
else:
    print("❌ 分析する結果がありません")

## 6. 可視化グラフ作成

In [ ]:
# グラフ作成
if classifier.results and len(classifier.results) > 1:
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 改善率分布
    valid_improvements = df[df['improvement_rate'] != 0]['improvement_rate']
    if len(valid_improvements) > 0:
        axes[0, 0].hist(valid_improvements, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('改善率分布')
        axes[0, 0].set_xlabel('改善率 (%)')
        axes[0, 0].set_ylabel('頻度')
        axes[0, 0].axvline(valid_improvements.mean(), color='red', linestyle='--', 
                          label=f'平均: {valid_improvements.mean():.1f}%')
        axes[0, 0].legend()
    
    # 2. カテゴリ別改善率
    category_improvement = df[df['improvement_rate'] != 0].groupby('semantic_category')['improvement_rate'].mean()
    if len(category_improvement) > 0:
        category_improvement.plot(kind='bar', ax=axes[0, 1], color='lightgreen')
        axes[0, 1].set_title('カテゴリ別平均改善率')
        axes[0, 1].set_xlabel('意味カテゴリ')
        axes[0, 1].set_ylabel('平均改善率 (%)')
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. 確信度比較
    valid_confidences = df[(df['general_confidence'].notna()) & (df['specialized_confidence'].notna())]
    if len(valid_confidences) > 0:
        axes[1, 0].scatter(valid_confidences['general_confidence'], 
                          valid_confidences['specialized_confidence'], alpha=0.7)
        axes[1, 0].plot([0, 1], [0, 1], 'r--', alpha=0.8)
        axes[1, 0].set_title('確信度比較')
        axes[1, 0].set_xlabel('汎用アプローチ確信度')
        axes[1, 0].set_ylabel('特化アプローチ確信度')
        axes[1, 0].set_xlim(0, 1)
        axes[1, 0].set_ylim(0, 1)
    
    # 4. カテゴリ分布
    category_counts = df['semantic_category'].value_counts()
    axes[1, 1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
    axes[1, 1].set_title('意味カテゴリ分布')
    
    plt.tight_layout()
    plt.show()
    
    # 統計的検定
    if len(valid_confidences) > 1:
        t_stat, p_value = stats.ttest_rel(valid_confidences['specialized_confidence'], 
                                         valid_confidences['general_confidence'])
        print(f"\n📊 統計的検定結果:")
        print(f"  - t統計量: {t_stat:.4f}")
        print(f"  - p値: {p_value:.4f}")
        print(f"  - 有意性: {'有意 (p < 0.05)' if p_value < 0.05 else '非有意 (p >= 0.05)'}")

else:
    print("❌ グラフ作成に十分なデータがありません")

## 7. 結果のダウンロード

In [ ]:
# 結果をファイルに保存
if classifier.results:
    # CSV形式で保存
    df.to_csv('research_results.csv', index=False, encoding='utf-8')
    
    # JSON形式で保存
    with open('research_results.json', 'w', encoding='utf-8') as f:
        json.dump(classifier.results, f, ensure_ascii=False, indent=2)
    
    # 統計サマリー保存
    summary = {
        'total_images': len(df),
        'categories_detected': df['semantic_category'].nunique(),
        'average_improvement': valid_improvements.mean() if len(valid_improvements) > 0 else 0,
        'improvement_std': valid_improvements.std() if len(valid_improvements) > 0 else 0,
        'category_distribution': df['semantic_category'].value_counts().to_dict(),
        'statistical_test': {
            't_statistic': t_stat if 't_stat' in locals() else None,
            'p_value': p_value if 'p_value' in locals() else None
        }
    }
    
    with open('research_summary.json', 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    
    print("✅ 結果ファイル保存完了:")
    print("  - research_results.csv")
    print("  - research_results.json")
    print("  - research_summary.json")
    
    # Colabからダウンロード
    if IN_COLAB:
        from google.colab import files
        files.download('research_results.csv')
        files.download('research_results.json')
        files.download('research_summary.json')
        print("✅ ファイルダウンロード完了")
    
    # Google Driveにもバックアップ
    if IN_COLAB and os.path.exists('/content/drive/MyDrive'):
        import shutil
        backup_dir = '/content/drive/MyDrive/research_backup'
        os.makedirs(backup_dir, exist_ok=True)
        
        shutil.copy('research_results.csv', backup_dir)
        shutil.copy('research_results.json', backup_dir)
        shutil.copy('research_summary.json', backup_dir)
        print("✅ Google Driveバックアップ完了")

else:
    print("❌ 保存する結果がありません")

## 8. 研究結果サマリー

In [ ]:
# 最終的な研究結果サマリー
print("="*60)
print("🎓 意味カテゴリ画像分類研究 - 最終結果")
print("="*60)

if classifier.results:
    print(f"\n📊 実験概要:")
    print(f"  - 処理画像数: {len(df)}")
    print(f"  - 検出カテゴリ数: {df['semantic_category'].nunique()}")
    print(f"  - 使用AI技術: BLIP, CLIP, YOLO, WordNet")
    
    if len(valid_improvements) > 0:
        print(f"\n🚀 特化アプローチの効果:")
        print(f"  - 平均改善率: {valid_improvements.mean():.2f}%")
        print(f"  - 改善を示した画像: {len(valid_improvements[valid_improvements > 0])}/{len(valid_improvements)}")
        print(f"  - 最大改善率: {valid_improvements.max():.2f}%")
        
        if 'p_value' in locals() and p_value < 0.05:
            print(f"  - 統計的有意性: 有意 (p = {p_value:.4f})")
            print(f"  - 結論: 特化アプローチは統計的に有意な改善を示した")
        else:
            print(f"  - 統計的有意性: 非有意 (p = {p_value:.4f})" if 'p_value' in locals() else "  - 統計的検定: データ不足")
    
    print(f"\n📈 カテゴリ別結果:")
    for category in df['semantic_category'].unique():
        cat_data = df[df['semantic_category'] == category]
        cat_improvements = cat_data[cat_data['improvement_rate'] != 0]['improvement_rate']
        if len(cat_improvements) > 0:
            print(f"  - {category}: {len(cat_data)}画像, 平均改善率 {cat_improvements.mean():.2f}%")
        else:
            print(f"  - {category}: {len(cat_data)}画像, 改善率データなし")
    
    print(f"\n📋 今後の研究方向:")
    print(f"  - より大規模なデータセットでの検証")
    print(f"  - 追加の意味カテゴリの探索")
    print(f"  - より高度な特化アプローチの開発")
    print(f"  - 実世界アプリケーションへの応用")
    
else:
    print("❌ 研究結果がありません。画像をアップロードして実験を実行してください。")

print("\n✅ 研究実験完了！")